In [5]:
import os
import re
import sys
import pandas as pd
from tqdm.auto import tqdm

In [6]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
from src.utils import load_dataset

df = load_dataset("df_with_llmkeyphrases")

Данные успешно загружены из ../data/df_with_llmkeyphrases.parquet. Количество записей: 200


In [8]:
from src.classic_keywords.keyphrase_extraction import extract_yake_keyphrases
from src.search_tf_idf.search import TfidfSearch
from src.classic_keywords.preprocessing import preprocess_text

pd.set_option('display.max_colwidth', 300)
tqdm.pandas()

In [9]:
from src.classic_keywords.preprocessing import lemmatize_word 

In [10]:
tfidf_search_engine = TfidfSearch()

tfidf_search_engine.build_index(
    lemmatized_texts=df['lemmatized_abstract'],
    original_texts=df['abstract']
)

Building TF-IDF index...
TF-IDF index built successfully.


In [11]:
def highlight_keywords(text: str, query: str, preprocessor_func) -> str:
    """
    Выделяет в оригинальном тексте слова, леммы которых присутствуют в запросе.
    """
    query_lemmas = set(preprocessor_func(query).split())
    
    tokens = re.split(r'(\s+)', text) 
    
    highlighted_tokens = []
    for token in tokens:
        if not token.strip():
            highlighted_tokens.append(token)
            continue
        
        cleaned_token = token.strip('.,!?:;()[]{}')
        
        if lemmatize_word(cleaned_token.lower()) in query_lemmas:
            highlighted_tokens.append(f"**{token}**")
        else:
            highlighted_tokens.append(token)
            
    return "".join(highlighted_tokens)

from IPython.display import display, Markdown

In [12]:
queries = [
    "статьи про графовые алгоритмы в компьютерном зрении",
    "модернизация производственных процессов на предприятии",
    "анализ социальных сетей и выявление сообществ"
]

for query in queries:
    print(f"--- Результаты поиска для запроса: '{query}' ---")
    
    search_results = tfidf_search_engine.search(query, preprocessor_func=preprocess_text)
    
    if not search_results:
        print("Ничего не найдено.")
    else:
        for i, (doc_index, doc_text) in enumerate(search_results):
            highlighted = highlight_keywords(doc_text, query, preprocess_text)
            md_output = f"""{i+1}. (Индекс документа: {doc_index})
            > {highlighted[:500]}..."""
            display(Markdown(md_output))

--- Результаты поиска для запроса: 'статьи про графовые алгоритмы в компьютерном зрении' ---


1. (Индекс документа: 83)
            > Предложен подход к реализации программного обеспечения для высокоточной трёхмерной визуализации резки заготовки на пятикоординатном фрезерном станке с ЧПУ. Рассматриваемое программное обеспечение позволяет моделировать пятикоординатную обработку заготовки разнообразными видами фрез под любым углом. Следует отметить, что в процессе разработки не использовались сторонние методики, все **алгоритмы** и подходы полностью авторские. Предлагаемое решение включает в себя: модель данных для эффективного ...

2. (Индекс документа: 17)
            > Представлен **алгоритм** работы **компьютерной** программы автоматизированного выбора фасовочно -упаковочных автоматов для сыпучих продуктов в мягкую тару. Данная программа содержит все необходимые базы данных и позволяет выбрать оптимальный дозатор и упаковочный материал для продукта. Выбор осуществляется на основе функционально стоимостного и критериального анализов....

3. (Индекс документа: 92)
            > Рассмотрен опыт выполнения учебно-исследовательской работы студентов с точки **зрения** проблемы соотношения, взаимодействия теории и эмпирии в социологическом исследовании. Последовательно раскрывается содержание этапов овладения студентами исследовательскими компетенциями как составной частью профессиональных компетенций выпускника....

4. (Индекс документа: 20)
            > Описан **алгоритм** генерации непротиворечивых решающих правил для выбора рациональной альтернативы из множества функциональных схем системы автоматизации с использованием математического аппарата теорий нечеткой логики и приближенных множеств. **Алгоритм** позволяет проводить многокритериальную оценку сложных вариантов схем без заметного увеличения временной сложности по сравнению с оценкой одноконтурных схем. Сформулированы требования к обучающим множествам функциональных схем с учетом особенн...

...

1. (Индекс документа: 59)
            > Рассматриваются искусственные нейроны, чьи весовые коэффициенты будут изменяться по специальному закону, основанному на интегрированном в их модели обратном распространении. Для этого коэффициенты погрешности обратного распространения вводятся в явном виде во все модели нейронов и осуществляется передача их значений вдоль межнейронных связей. В дополнение к этому вводится специальный тип нейронов с эталонными входами, которые будут выступать в качестве основного источника первичной оценки погреш...

2. (Индекс документа: 103)
            > Предложена математическая модель для автоматизации поиска и оценки аномалий объема сетевого трафика. Разработанная модель может быть использована в системе управления трафика для **анализа** состояния вычислительной **сети** с целью **выявления** неисправностей сетевого оборудования, **выявления** случайных и преднамеренных действий со стороны пользователей, а также действия злоумышленников. Предложенная схема может быть использована в системе управления трафика как внутренней, так и внешней **с...

3. (Индекс документа: 177)
            > В статье нашли отражение результаты проведенного автором исследования процесса решения **социальных** проблем в Дальневосточном регионе в 1918-1922 гг. На конкретных примерах рассмотрена специфика становления системы **социального** обеспечения населения, в том числе и военнослужащих, на Дальнем Востоке. Автор, на основе неопубликованных материалов, показывает, как государственные органы использовали **социальную** политику в качестве инструмента в политической борьбе. Рассмотренные в статье мат...

4. (Индекс документа: 183)
            > В статье развит методологический подход к оценке эффективности **социальной** политики, основанный на логике квази-экспериментального подхода, основой которого является поиск ответа на вопрос: «Что произошло бы с участником **социальной** программы, если бы он не стал объектом ее воздействия?»....

5. (Индекс документа: 36)
            > В статье представлена специфика использования электронных образовательных ресурсов в деятельности **социального** проекта, популяризирующего русский язык. Проект «Современный русский» работает в Омском государственном педагогическом университете более шести лет и оказывает лингвистическую помощь жителям 26 стран мира. В проекте действуют девять направлений практической филологии, что делает его весьма востребованным среди различных возрастных и **социальных** групп...

* Как видно из подсветки, TF-IDF успешно находит документы, содержащие точные ключевые слова из запроса, такие как 'модернизация' и 'производственных'. Ранжирование происходит на основе частоты и редкости этих слов.
* Однако этот подход полностью игнорирует семантику. Если в запросе указано 'модернизация производства', а в тексте — 'оптимизация технологических процессов на заводе', TF-IDF, скорее всего, не найдет этот документ или даст ему низкий ранг.